In [243]:
import connection as cn
import pandas as pd

from sqlalchemy import create_engine
from datetime import datetime

In [244]:
engine = create_engine(f"postgresql+psycopg2://{cn.user}:{cn.password}@{cn.localhost}/{cn.database}",echo=False)

In [ ]:
booking = pd.read_sql("select * from booking", engine)
region = pd.read_sql("select * from region",engine)
trailer = pd.read_sql("select * from trailer",engine)
destination = pd.read_sql("select * from destination",engine)
province = pd.read_sql("select * from province_entity",engine)
detail = pd.read_sql("select * from detail",engine).rename(columns={"id": "detail_id"})
user = pd.read_sql("select * from users",engine)
vehicle = pd.read_sql("select * from vehicle",engine)
current_date = datetime.now().date()

In [246]:
merged_df = booking.merge(region, left_on='fromId', right_on='id').merge(region, left_on='toId', right_on="id").drop(['id_y','id','fromId','toId'],axis=1).merge(trailer, left_on="trailerId", right_on="id").drop(['createDate_y','actExpire','id','trailerId'],axis=1).rename(columns={"region_x": "from","region_y": "to", "createDate_x": "createDate","registration":"trailer_registration"})

In [247]:
merged_df.createDate = merged_df.createDate.dt.date
merged_df.departureDate = merged_df.departureDate.dt.date
merged_df.arriveDate = merged_df.arriveDate.dt.date


In [248]:
merged_df = merged_df.astype({"createDate": "datetime64[ns]", 
                              "departureDate": "datetime64[ns]",
                              "arriveDate": "datetime64[ns]",
                              "status":"category",
                              "capacity":"Int8",
                              "from": "category",
                              "to":"category",
                              "deposit": "Int16",
                              "trailer_registration": "category"
                              }).rename(columns={"id_x": "booking_id"})



In [249]:

get_route  = (destination.merge(booking,left_on='booking', right_on="id").merge(province, left_on="province", right_on='id')).reset_index().groupby('booking').apply(lambda x: ','.join(x.province_y)).reset_index().rename(columns={0: 'route'});


C:\Users\User\AppData\Local\Temp\ipykernel_26504\731876730.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  get_route  = (destination.merge(booking,left_on='booking', right_on="id").merge(province, left_on="province", right_on='id')).reset_index().groupby('booking').apply(lambda x: ','.join(x.province_y)).reset_index().rename(columns={0: 'route'});


In [250]:
merged_df = merged_df.merge(get_route, left_on="booking_id", right_on='booking').sort_values('departureDate',ascending=False).drop(columns=["createDate"],axis=1)


In [251]:

detail = detail.merge(merged_df, left_on="bookingId", right_on="booking_id").drop(columns=['bookingId','deposit','booking']).merge(vehicle, left_on="vehicleId", right_on="id").drop(columns=["ownerId",'id',"vehicleId",'booking_id']).merge(
    user,left_on='userId', right_on="id").drop(columns=['createDate','age','userId','id']).merge(province,left_on="pickId",right_on="id").drop(columns=['id']).rename(columns={"province": "pick"}).merge(province, left_on="dropId",right_on='id').drop(columns=['id','pickId','dropId','capacity']).rename(columns={"province":"drop"})



In [252]:
details = detail.rename(columns={"from": "froms"}).sort_values('departureDate', ascending=False).drop(columns=["arriveDate","route",'sex']).query('status == "pending"').rename(columns={"tel":"telephone Number"})
# detail

In [253]:
bookings  = merged_df.query("status =='pending' | status =='depart'").sort_values(["departureDate"], ascending=True).assign(isChiangmai = lambda x: x.route.str.contains('chiangmai')).drop(columns=['booking'])

In [254]:
north = bookings.query("route.str.contains('chiangrai|chiangmai', regex=True,case=False)").sort_values(['departureDate'], ascending=True).drop(columns=['route']).set_index("booking_id")

chiangmai = north.query('trailer_registration == "outsource_petch"')
chiangrai = north.query('trailer_registration == "outsource_one"')


In [255]:
chiangrai

,departureDate,arriveDate,capacity,status,deposit,from,to,trailer_registration,isChiangmai
booking_id,,,,,,,,,
31,2025-09-02,2025-09-04,2,pending,1000,central,chiangrai,outsource_one,False
32,2025-09-07,2025-09-09,0,pending,0,central,chiangrai,outsource_one,False


In [256]:
chiangmai

,departureDate,arriveDate,capacity,status,deposit,from,to,trailer_registration,isChiangmai
booking_id,,,,,,,,,
48,2025-09-02,2025-09-03,0,pending,0,central,chiangmai,outsource_petch,True
34,2025-09-05,2025-09-06,1,pending,500,central,chiangmai,outsource_petch,True
49,2025-09-09,2025-09-10,0,pending,0,central,chiangmai,outsource_petch,True
50,2025-09-12,2025-09-13,0,pending,0,central,chiangmai,outsource_petch,True
51,2025-09-16,2025-09-17,0,pending,0,central,chiangmai,outsource_petch,True
52,2025-09-19,2025-09-20,0,pending,0,central,chiangmai,outsource_petch,True
53,2025-09-23,2025-09-24,0,pending,0,central,chiangmai,outsource_petch,True
54,2025-09-26,2025-09-27,0,pending,0,central,chiangmai,outsource_petch,True
55,2025-09-30,2025-10-01,0,pending,0,central,chiangmai,outsource_petch,True


In [257]:
south = bookings.query("route.str.contains('hatyai|phuket', regex=True,case=False)").sort_values(['from', 'to','departureDate']).drop(columns=['isChiangmai','route']).set_index("booking_id")
south

,departureDate,arriveDate,capacity,status,deposit,from,to,trailer_registration
booking_id,,,,,,,,
28,2025-08-31,2025-09-02,7,depart,3000,central,south,722735
39,2025-09-10,2025-09-12,2,pending,1000,central,south,722735
40,2025-09-16,2025-09-18,1,pending,500,central,south,722735
41,2025-09-22,2025-09-24,0,pending,0,central,south,722735
42,2025-09-28,2025-09-30,0,pending,0,central,south,722735
43,2025-10-03,2025-10-05,0,pending,0,central,south,722735
38,2025-09-02,2025-09-04,8,pending,4000,south,central,722735
44,2025-09-12,2025-09-14,3,pending,1500,south,central,722735
45,2025-09-18,2025-09-20,0,pending,0,south,central,722735


In [258]:
isan = bookings.query('trailer_registration ==  "outsource_ta"').drop(columns=['isChiangmai','route']).set_index('booking_id')

isan

,departureDate,arriveDate,capacity,status,deposit,from,to,trailer_registration
booking_id,,,,,,,,


In [259]:

pending = details.sort_values(["departureDate","froms","name"]).set_index("detail_id").drop(columns=['createDate_x','createDate_y'])
pending


,remark,departureDate,status,froms,to,trailer_registration,brand,registration,model,name,lastName,telephone Number,pick,drop
detail_id,,,,,,,,,,,,,,
41,,2025-09-02,pending,central,chiangrai,outsource_one,mg,กต3148,mg3,จารุวรรณ,ศักดิ์สูง,0804094542,bangkok,phayao
76,None,2025-09-02,pending,central,chiangrai,outsource_one,toyota,ขก2959,altis,วัชรพงศ์,ปวงแก้ว,0897554664,bangkok,chiangrai
63,None,2025-09-02,pending,south,central,722735,toyota,ขฉ2646,sienta,ธนัญชกาญจน์,สวัสดิรัชตภักดิ์,0864245626,phuket,bangkok
64,None,2025-09-02,pending,south,central,722735,toyota,ขฉ2744,sienta,ธนัญชกาญจน์,สวัสดิรัชตภักดิ์,0864245626,phuket,bangkok
67,None,2025-09-02,pending,south,central,722735,nissan,ญธ1834,march,ธนัทเดช,ทองแวว,0810837996,phuket,bangkok
58,เสาวลักษณ์ 0818985202 เบอร์ผู้ส่ง,2025-09-02,pending,south,central,722735,toyota,1ขง1210,yaris,พชรมน,สุวรรณรัตน์,0828262008,hatyai,bangkok
57,None,2025-09-02,pending,south,central,722735,landrover,บน5678,None,ภูริพงศ์,มังกรกนก,0843317070,hatyai,bangkok
31,None,2025-09-02,pending,south,central,722735,benz,5กส6111,E220,วรรณกร,ชิณวงศ์,0902954635,phuket,bangkok
60,None,2025-09-02,pending,south,central,722735,honda,8กฒ7738,hrv,ศิริวรรณ,พรหมจันทร์,0644306310,phuket,bangkok


In [260]:
today_depart = detail.query("status == 'depart' and departureDate.dt.date <= @current_date").drop(columns=['arriveDate','route','sex','createDate_x','createDate_y']).set_index('detail_id').sort_values(['name','pick']).rename(columns={"tel":"telephone number"})
today_depart

,remark,departureDate,status,from,to,trailer_registration,brand,registration,model,name,lastName,telephone number,pick,drop
detail_id,,,,,,,,,,,,,,
66,เบอร์โทรผู้ส่ง 0917799797,2025-08-31,depart,central,south,722735,benz,วบ288,None,ชานนทร์,ศิริกุลสถิตย์,0917799797,bangkok,nakhonsi
72,0656429451 เบอร์คนส่งรถเวลา 10:00,2025-08-31,depart,central,south,722735,mg,กล3796,mg3,ทรงยศ,ทับทอง,0897253234,bangkok,surat
71,นัดรับแถวมหิดล 0653945950,2025-08-31,depart,central,south,722735,mg,7กบ1856,mg3,ทรงยศ,ทับทอง,0897253234,nakhonpathom,surat
56,ลูกค้ามาส่งรถ 08:30,2025-08-31,depart,central,south,722735,honda,ฎฎ15,jazz,นิพนธ์,กิติสุข,0818588899,bangkok,phuket
74,ซาฟารีคาร์เซ็นเตอร์ ชื่อเต้น บังฟารี ลงหน้าเต้น,2025-08-31,depart,central,south,722735,honda,2ขฒ6574,hrv,บัง,ซาฟารี,0817665074,bangkok,hatyai
61,,2025-08-31,depart,central,south,722735,mitsubishi,ขก3016,triton,มนัสนันท์,คอนเนลลี่,0909717143,bangkok,phuket
59,ผู้รับ 095-8795928,2025-08-31,depart,central,south,722735,toyota,6ขร8113,altis,ศศิกานต์,ถิรโสภี,0929056591,bangkok,phuket


In [261]:
south_date = current_date - pd.Timedelta(days=1)


arrived = detail.query('status == "arrived" & arriveDate.dt.date == @current_date or arriveDate.dt.date == @south_date').drop(columns=['route','departureDate','sex']).set_index('detail_id')
arrived



,createDate_x,remark,arriveDate,status,from,to,trailer_registration,createDate_y,brand,registration,model,name,lastName,tel,pick,drop
detail_id,,,,,,,,,,,,,,,,


In [262]:
today_booking = details.drop(columns=['createDate_y']).set_index('detail_id').rename(columns={"createDate_x": "createDate"}).query('createDate.dt.date == @current_date')
today_booking

,createDate,remark,departureDate,status,froms,to,trailer_registration,brand,registration,model,name,lastName,telephone Number,pick,drop
detail_id,,,,,,,,,,,,,,,
77,2025-09-01 11:13:31,None,2025-09-10,pending,central,south,722735,honda,4ขท4249,city,พัชรี,วงษ์บุญ,0615354924,bangkok,phuket
76,2025-09-01 10:42:40,None,2025-09-02,pending,central,chiangrai,outsource_one,toyota,ขก2959,altis,วัชรพงศ์,ปวงแก้ว,0897554664,bangkok,chiangrai


In [263]:
user.set_index('id').sort_index().tail(5)

,name,lastName,tel,sex,age,createDate
id,,,,,,
75,อนุวัฒน์,ปานตุ่น,0810808980,male,18,2025-08-31 01:31:40
76,บัง,ซาฟารี,0817665074,male,18,2025-08-31 10:43:01
77,ขนิษฐา,กีรติภัทรกาญจน์,0862685544,female,18,2025-08-31 19:56:45
78,วัชรพงศ์,ปวงแก้ว,0897554664,male,38,2025-09-01 10:42:40
79,พัชรี,วงษ์บุญ,0615354924,female,18,2025-09-01 11:13:31


In [264]:
user_car =vehicle.merge(user, left_on="ownerId", right_on="id").set_index('id_x').drop( columns=['createDate_x','createDate_y','id_y','sex','age','ownerId','tel']).sort_index().tail(5)
user_car

,brand,registration,model,name,lastName
id_x,,,,,
63,suzuki,4กข9983,swift,อนุวัฒน์,ปานตุ่น
64,honda,2ขฒ6574,hrv,บัง,ซาฟารี
65,volvo,กม6373,V50,ขนิษฐา,กีรติภัทรกาญจน์
66,toyota,ขก2959,altis,วัชรพงศ์,ปวงแก้ว
67,honda,4ขท4249,city,พัชรี,วงษ์บุญ


In [265]:
# chiangmai = detail.query("route.str.contains('chiangrai|chiangmai', regex=True,case=False)").assign(isChiangmai = lambda x: x.route.str.contains('chiangmai'))
# chiangmai.groupby(["isChiangmai", pd.Grouper(key='departureDate', freq='W-SUN')]).count()

In [266]:
# test = detail.groupby(['from', pd.Grouper(key='departureDate', freq='W-MON'),'to']).count()



In [267]:
# bookings.capacity.sum()

In [268]:
# with pd.ExcelWriter('booking.xlsx', engine='xlsxwriter',datetime_format='yyyy-mm-dd') as writer:
#     chiangmai.to_excel(writer, sheet_name="chiangmai_booking",index=False, startrow=2)
#     chiangrai.to_excel(writer, sheet_name="chiangrai_booking", index=False, startrow=2)
#     south.to_excel(writer,sheet_name="south_booking",index=False, startrow=2)
#     isan.to_excel(writer,sheet_name="isan_booking",index=False, startrow=2)
#     pending.to_excel(writer,sheet_name="detail_booking",index=False, startrow=2)
#     today_depart.to_excel(writer,sheet_name="depart_booking",index=False, startrow=2)
#     arrived.to_excel(writer,sheet_name="arrived_booking",index=False, startrow=2)
#     writer.sheets["chiangmai_booking"].autofit()
#     writer.sheets["chiangrai_booking"].autofit()
#     writer.sheets["isan_booking"].autofit()
#     writer.sheets['south_booking'].autofit()
#     writer.sheets['detail_booking'].autofit()
#     writer.sheets['depart_booking'].autofit()
#     writer.sheets['arrived_booking'].autofit()


In [269]:
# detail.query(f"status == 'depart' and departureDate.dt.date == @current_date").reset_index().drop(["index"],axis=1)
# weekly report
# detail.loc[:, ['createDate_x','departureDate','arriveDate',"from","to",'pick','drop']].groupby(['from','to']).resample('W-Mon',on="createDate_x").count()

In [270]:
# user.sex.value_counts().plot.pie(y='sex',startangle=90)

In [271]:
# user.query('age > 18').age.value_counts().sort_index().plot.bar(color="green",title ="age")

In [272]:
# booking.drop(columns=['createDate']).sort_values('departureDate')

# detail.drop(columns=['remark', 'createDate_x', 'arriveDate','trailer_registration','route','createDate_y','from','to','registration','name','lastName','tel','departureDate','detail_id','status']).groupby(['pick','drop']).count()